In [1]:
import json
import random
from geopy.distance import geodesic
import numpy as np
import datetime

def calculate_average_location(coords):
    if not coords:
        return None

    sum_lat = 0
    sum_lon = 0
    count = 0

    for coord in coords:
        if coord and len(coord) == 2:
            sum_lat += coord[0]
            sum_lon += coord[1]
            count += 1

    if count == 0:
        return None

    return [sum_lat / count, sum_lon / count]

def calculate_condition_value(activity):
    # Extracting start and end coordinates
    start_coords = activity.get('start_latlng', [])
    end_coords = activity.get('end_latlng', [])
    # Combining the start and end coordinates for distance calculation 
    coords = []
    if start_coords:
        coords.append(start_coords)
    if end_coords:
        coords.append(end_coords)

    if not coords:
        return None

    avg_heart_rate = activity['average_heartrate']
    avg_speed = activity['average_speed']*3.6
    # heart_rate_variability = np.std(activity['heart_rate'])

    if avg_heart_rate == 0:
        return None

    distance = activity['distance'] / 1000
    # start_time = datetime.datetime.fromtimestamp(activity['timestamp'][0])
    # end_time = datetime.datetime.fromtimestamp(activity['timestamp'][-1])
    # duration = (end_time - start_time).total_seconds() / 3600
    duration = activity['elapsed_time'] / 3600
    if duration == 0:
        return None

    condition_value = (avg_speed * distance) / (avg_heart_rate * duration)
    return condition_value

file_path = '../Data/Andere/filtered_strava_document.json'
with open(file_path, 'r') as f:
    data = json.load(f)


# Initialize lists to store activity data
activity_ids = []
speeds = []
distances = []
average_heartrates = []
altitude_changes = []
locations = []
condition_values = []

for activity in data:
    # Filter alleen activiteiten van het type "Run"
    if activity.get('sport_type') == 'Run':
        activity_ids.append(activity['id'])
        speeds.append(activity.get('average_speed', 0) * 3.6)
        distances.append(activity.get('distance', 0) / 1000.0)
        average_heartrates.append(activity.get('average_heartrate', 0))
        altitude_changes.append(activity.get('total_elevation_gain', 0))

        coords = activity.get('start_latlng', [])
        if coords:
            locations.append(coords)
        condition_value = calculate_condition_value(activity)
        if condition_value:
            condition_values.append(condition_value)

average_speed = np.mean(speeds) if speeds else 0
std_speed = np.std(speeds) if speeds else 0
average_distance = np.mean(distances) if distances else 0
std_distance = np.std(distances) if distances else 0
average_average_heartrate = np.mean(average_heartrates) if average_heartrates else 0
std_average_heartrate = np.std(average_heartrates) if average_heartrates else 0
average_altitude_change = np.mean(altitude_changes) if altitude_changes else 0
std_altitude_change = np.std(altitude_changes) if altitude_changes else 0
average_condition_value = np.mean(condition_values) if condition_values else 0
std_condition_value = np.std(condition_values) if condition_values else 0


# Calculate average location
average_location = calculate_average_location(locations) if locations else [0, 0]

# Calculate weights
total_std = (1/(std_distance+0.0001)) + (1/(std_altitude_change+0.0001)) + (1/(std_condition_value+0.0001))
weight_std_distance = (1/(std_distance+0.0001)) / total_std
weight_std_altitude_change = (1/(std_altitude_change+0.0001)) / total_std
weight_std_condition_value = (1/(std_condition_value+0.0001)) / total_std

# Construct the final JSON object
result_json = {
    "userId": 1,
    "activity_ids": activity_ids,
    "speeds": speeds,
    "distances": distances,
    "average_heartrates": average_heartrates,
    "altitude_changes": altitude_changes,
    "locations": locations,
    "condition_values": condition_values,
    "average_location": average_location,
    "average_speeds": average_speed,
    "std_speeds": std_speed,
    "average_distances": average_distance,
    "std_distances": std_distance,
    "average_average_heartrates": average_average_heartrate,
    "std_average_heartrates": std_average_heartrate,
    "average_altitude_changes": average_altitude_change,
    "std_altitude_changes": std_altitude_change,
    "weight_std_distance": weight_std_distance,
    "weight_std_altitude_change": weight_std_altitude_change,
    "weight_std_condition_value": weight_std_condition_value,
    "average_condition_value": average_condition_value,
    "std_condition_values": std_condition_value
}
output_file_path = '../Data/Andere/user_profile_strava.json'

# Sla het resultaat op in een JSON-bestand
with open(output_file_path, 'w') as outfile:
    json.dump(result_json, outfile)

print(f"Gebruikersprofiel opgeslagen als {output_file_path}")

Gebruikersprofiel opgeslagen als ../Data/Andere/user_profile_strava.json
